### Production Features Pipeline - CSV Version

This notebook is run daily from a Github Action. 

1. It scrapes the results from the previous day's games, performs feature engineering, and saves the results back to a csv file. This is an alternative version of the pipeline that DOES NOT utilize the Hopsworks.ai Feature Store and is less dependent on other platforms.

2. It scrapes the upcoming games for today, and saves the blank records back into the csv file so that they can be accessed by the model for the prediction.

**Note:**
There are two options for webscraping in this notebook. 
Set the 'WEBSCRAPER' variable to either 'SCRAPINGANT' or 'SELENIUM' to choose which version to run.

1. SCRAPINGANT: Uses a webscraping service with a Python API, ScrapingAnt, which handles all the proxy server issues, but does require an account. The free account allows for 1000 page requests, which is more than enough for this project. Proxies are required when running this notebook from a Github Action or otherwise key data will fail to be scraped from NBA.com. 

2. SELENIUM: This option does not currently integrate proxy servers into the webscraping process, which can cause issues when scraping from certain locations, in particular Github Actions. For occasional use from local machines, this option may work fine, but you may need to setup a proxy server.

In [1]:
# select web scraper; 'SCRAPINGANT' or 'SELENIUM'
# SCRAPINGANT requires a subscription but includes a proxy server

WEBSCRAPER = 'SCRAPINGANT'
#WEBSCRAPER = 'SELENIUM'

In [2]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

import json

import time

from pathlib import Path  #for Windows/Linux compatibility

# change working directory to project root when running from notebooks folder to make it easier to import modules
# and to access sibling folders
os.chdir('..') 

 
from src.webscraping import (
    get_new_games,
    activate_web_driver,
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.dashboard_processing import (
    NBADataProcessor,
)

from src.google_drive_utils import (
    upload_to_drive,
)

DATAPATH = Path(r'data')
GOOGLE_FOLDER_ID = "1y5AfF3KZ8FGzxr2pyuncXJpKWEa5j-CL"

**Load API keys**

In [3]:
from dotenv import load_dotenv

load_dotenv()

#try:
#    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
#except:
#    raise Exception('Set environment variable HOPSWORKS_API_KEY')


# if scrapingant is chosen then set the api key, otherwise load the selenium webdriver
if WEBSCRAPER == 'SCRAPINGANT':
    try:
        SCRAPINGANT_API_KEY = os.environ['SCRAPINGANT_API_KEY']
    except:
        raise Exception('Set environment variable SCRAPINGANT_API_KEY')
    driver = None
    
elif WEBSCRAPER == 'SELENIUM':
    driver = activate_web_driver('chromium')
    SCRAPINGANT_API_KEY = ""
    



**Scrape New Completed Games and Format Them**

In [4]:


df_new = get_new_games(SCRAPINGANT_API_KEY, driver)

if df_new.empty:
    print('No new games to process')

    # determine what season we are in currently
    today = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    if today.month >= 10:
        SEASON = today.year
    else:
        SEASON = today.year - 1
else:

    # get the SEASON of the last game in the database
    # this will used when constructing rows for prediction
    SEASON = df_new['SEASON'].max()

    df_new




Current month is 10
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Regular+Season&DateFrom=10/15/25&DateTo=10/22/25
No new games to process


**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
matchups, game_ids = get_todays_matchups(SCRAPINGANT_API_KEY, driver)

if matchups is None:
    print('No games today')
else:
    print(matchups)
    print(game_ids)


Wed
[['1610612739', '1610612752'], ['1610612751', '1610612766'], ['1610612748', '1610612753'], ['1610612761', '1610612737'], ['1610612755', '1610612738'], ['1610612765', '1610612741'], ['1610612740', '1610612763'], ['1610612764', '1610612749'], ['1610612746', '1610612762'], ['1610612759', '1610612742'], ['1610612758', '1610612756'], ['1610612750', '1610612757']]
['22500003', '22500080', '22500081', '22500082', '22500083', '22500084', '22500085', '22500086', '22500087', '22500004', '22500088', '22500089']


**Close Webdriver**

In [6]:
if WEBSCRAPER == 'SELENIUM':
    driver.close() 

**Check if anything is going on in the season**

In [7]:
if (df_new.empty) and (matchups is None):
    print('No new games to process')
    #exit()
    

**Create Rows for Today's Games with Empty Stats**

In [8]:
# reformat today's matchups to the new games dataframe

if matchups is None:
    print('No games going on. Nothing to do.')
    #exit()    

else:

    df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
    for i, matchup in enumerate(matchups):
        game_details = {'HOME_TEAM_ID': matchup[1], 
                        'VISITOR_TEAM_ID': matchup[0], 
                        'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                        'GAME_ID': int(game_ids[i]),                       
                        'SEASON': SEASON,
                        } 
        game_details_df = pd.DataFrame(game_details, index=[i])
        # append to new games dataframe
        df_today = pd.concat([df_today, game_details_df], ignore_index = True)

    #blank rows will be filled with 0 to prevent issues with feature engineering
    df_today = df_today.fillna(0) 

    df_today



**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [9]:


df_old = pd.read_csv(DATAPATH / 'games.csv')

df_old


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,104.0,0.398,0.760,...,23.0,53.0,1.610613e+09,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,101.0,0.443,0.933,...,20.0,46.0,1.610613e+09,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,108.0,0.412,0.813,...,28.0,52.0,1.610613e+09,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,122.0,0.484,0.933,...,33.0,55.0,1.610613e+09,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,115.0,0.551,0.750,...,32.0,39.0,1.610613e+09,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30432,2025-05-23 00:00:00,42400302.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,109.0,47.600,85.700,...,18.0,38.0,NaN,114.0,51.800,71.400,43.300,26.0,32.0,0
30433,2025-05-22 00:00:00,42400312.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,118.0,50.000,79.200,...,30.0,42.0,NaN,103.0,41.400,76.900,28.200,20.0,41.0,1
30434,2025-05-21 00:00:00,42400301.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,135.0,51.100,70.000,...,24.0,46.0,NaN,138.0,51.000,75.000,40.500,26.0,39.0,0
30435,2025-05-20 00:00:00,42400311.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,114.0,50.000,80.800,...,27.0,46.0,NaN,88.0,34.900,71.400,29.400,18.0,42.0,1


**Update Yesterday's Matchup Predictions with New Final Results**

In [10]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results


# one approach is to simply drop the rows that were used for prediction yesterday
# which are games that have 0 points for home team
# and then append the new rows to the dataframe
df_old = df_old[df_old['PTS_home'] != 0]
df_old = pd.concat([df_old, df_new], ignore_index = True)


# save the new games to the database
df_old.to_csv(DATAPATH / 'games.csv', index=False)

df_old

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,104.0,0.398,0.760,...,23.0,53.0,1.610613e+09,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,101.0,0.443,0.933,...,20.0,46.0,1.610613e+09,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,108.0,0.412,0.813,...,28.0,52.0,1.610613e+09,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,122.0,0.484,0.933,...,33.0,55.0,1.610613e+09,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,115.0,0.551,0.750,...,32.0,39.0,1.610613e+09,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30432,2025-05-23 00:00:00,42400302.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,109.0,47.600,85.700,...,18.0,38.0,NaN,114.0,51.800,71.400,43.300,26.0,32.0,0
30433,2025-05-22 00:00:00,42400312.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,118.0,50.000,79.200,...,30.0,42.0,NaN,103.0,41.400,76.900,28.200,20.0,41.0,1
30434,2025-05-21 00:00:00,42400301.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,135.0,51.100,70.000,...,24.0,46.0,NaN,138.0,51.000,75.000,40.500,26.0,39.0,0
30435,2025-05-20 00:00:00,42400311.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,114.0,50.000,80.800,...,27.0,46.0,NaN,88.0,34.900,71.400,29.400,18.0,42.0,1


**Add Today's Matchups for Feature Engineering**

In [11]:
if matchups is None:
    print('No games today')
    df_combined = df_old
else:
    df_combined = pd.concat([df_old, df_today], ignore_index = True)
    df_combined

**Data Processing**

In [12]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF,TARGET
0,2022-03-12,22101005.0,1610612748.0,1610612750.0,2021.0,104.0,0.398,0.760,0.333,23.0,53.0,113.0,0.422,0.875,0.357,21.0,46.0,0.0,0,0.0
1,2022-03-12,22101006.0,1610612741.0,1610612739.0,2021.0,101.0,0.443,0.933,0.429,20.0,46.0,91.0,0.419,0.824,0.208,19.0,40.0,1.0,0,1.0
2,2022-03-12,22101007.0,1610612759.0,1610612754.0,2021.0,108.0,0.412,0.813,0.324,28.0,52.0,119.0,0.489,1.000,0.389,23.0,47.0,0.0,0,0.0
3,2022-03-12,22101008.0,1610612744.0,1610612749.0,2021.0,122.0,0.484,0.933,0.400,33.0,55.0,109.0,0.413,0.696,0.386,27.0,39.0,1.0,0,1.0
4,2022-03-12,22101009.0,1610612743.0,1610612761.0,2021.0,115.0,0.551,0.750,0.407,32.0,39.0,127.0,0.471,0.760,0.387,28.0,50.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30444,2025-10-22,22500086.0,1610612749,1610612764,2025.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN
30445,2025-10-22,22500087.0,1610612762,1610612746,2025.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN
30446,2025-10-22,22500004.0,1610612742,1610612759,2025.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN
30447,2025-10-22,22500088.0,1610612756,1610612758,2025.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN


**Feature Engineering**

In [13]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

# check that there are no NaN values
if df_combined.isnull().values.any():
    print('Warning: NaN values found in dataframe before feature engineering')
    print(df_combined[df_combined.isnull().any(axis=1)])
    df_combined = df_combined.fillna(0)

df_combined = process_features(df_combined)

#fix type conversion issues with hopsworks
df_combined['TARGET'] = df_combined['TARGET'].astype('int16')
df_combined['HOME_TEAM_WINS'] = df_combined['HOME_TEAM_WINS'].astype('int16')

# save file
df_combined.to_csv(DATAPATH / 'games_engineered.csv', index=False)


df_combined


      GAME_DATE_EST     GAME_ID HOME_TEAM_ID VISITOR_TEAM_ID  SEASON  \
30437    2025-10-22  22500003.0   1610612752      1610612739  2025.0   
30438    2025-10-22  22500080.0   1610612766      1610612751  2025.0   
30439    2025-10-22  22500081.0   1610612753      1610612748  2025.0   
30440    2025-10-22  22500082.0   1610612737      1610612761  2025.0   
30441    2025-10-22  22500083.0   1610612738      1610612755  2025.0   
30442    2025-10-22  22500084.0   1610612741      1610612765  2025.0   
30443    2025-10-22  22500085.0   1610612763      1610612740  2025.0   
30444    2025-10-22  22500086.0   1610612749      1610612764  2025.0   
30445    2025-10-22  22500087.0   1610612762      1610612746  2025.0   
30446    2025-10-22  22500004.0   1610612742      1610612759  2025.0   
30447    2025-10-22  22500088.0   1610612756      1610612758  2025.0   
30448    2025-10-22  22500089.0   1610612757      1610612750  2025.0   

       PTS_home  FG_PCT_home  FT_PCT_home  FG3_PCT_home  AST_ho

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,2003-10-28 00:00:00+00:00,20300001,1610612755,1610612748,2003,89,0.439941,0.533203,0.350098,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-28 00:00:00+00:00,20300003,1610612747,1610612742,2003,109,0.505859,0.600098,0.350098,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-28 00:00:00+00:00,20300002,1610612759,1610612756,2003,83,0.425049,0.769043,0.099976,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-29 00:00:00+00:00,20300006,1610612740,1610612737,2003,88,0.323975,0.700195,0.160034,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-29 00:00:00+00:00,20300007,1610612761,1610612751,2003,90,0.425049,0.799805,0.166992,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28318,2025-10-22 00:00:00+00:00,22500080,1610612766,1610612751,2025,0,0.000000,0.000000,0.000000,0,...,-2.517187,0.142708,-1.666667,-5.428571,-2.4,-1.266667,5.333333,3.285714,4.1,1.866667
28319,2025-10-22 00:00:00+00:00,22500084,1610612741,1610612765,2025,0,0.000000,0.000000,0.000000,0,...,1.662500,4.308333,4.000000,4.428571,4.7,6.333333,5.333333,5.571429,7.3,4.333333
28320,2025-10-22 00:00:00+00:00,22500089,1610612757,1610612750,2025,0,0.000000,0.000000,0.000000,0,...,-5.530469,-2.600521,2.333333,-2.000000,-0.7,0.066667,6.000000,9.142857,3.6,4.000000
28321,2025-10-22 00:00:00+00:00,22500082,1610612737,1610612761,2025,0,0.000000,0.000000,0.000000,0,...,-0.095313,2.662500,-2.333333,1.000000,-0.8,1.533333,-3.333333,1.571429,-1.7,-3.000000


**Process Data for Convenient Dashboarding**

In [14]:
processor = NBADataProcessor()
exported_files = processor.export_data_for_dashboard()

print("\nData Processing Complete!")
print(f"Files exported for Dashboards:")
for key, value in exported_files.items():
    if value:
        print(f"- {key}: {value}")

2025-10-22 11:26:30,474 INFO: Initialized NBADataProcessor with data_path=data, model_path=models
2025-10-22 11:26:30,475 INFO: Exporting data for dashboards to data
2025-10-22 11:26:30,475 INFO: Preparing today's games data
2025-10-22 11:26:30,476 INFO: Loading data from data\games_engineered.csv
2025-10-22 11:26:31,232 INFO: Successfully loaded data: 28323 rows, 245 columns
2025-10-22 11:26:31,236 INFO: Season selection -> upcoming: 2025, completed: 2024
2025-10-22 11:26:31,237 INFO: Filtering for season: 2025
2025-10-22 11:26:31,238 INFO: Processing data for prediction
Converting field PTS_home to int16 int64 0 0
Converting field AST_home to int16 int64 0 0
Converting field REB_home to int16 int64 0 0
Converting field PTS_away to int16 int64 0 0
Converting field AST_away to int16 int64 0 0
Converting field REB_away to int16 int64 0 0
2025-10-22 11:26:31,276 INFO: Making predictions
2025-10-22 11:26:31,277 WARNING: Model not loaded, loading now...
2025-10-22 11:26:31,278 INFO: Loadin

2025-10-22 11:26:32,190 INFO: Successfully loaded model
2025-10-22 11:26:32,191 INFO: Removing unused features
2025-10-22 11:26:32,546 INFO: Successfully made predictions for 12 games
2025-10-22 11:26:32,549 INFO: Prepared predictions for 12 games today
2025-10-22 11:26:32,551 INFO: Preparing processed games data
2025-10-22 11:26:32,552 INFO: Loading data from data\games_engineered.csv
2025-10-22 11:26:33,252 INFO: Successfully loaded data: 28323 rows, 245 columns
2025-10-22 11:26:33,254 INFO: Season selection -> upcoming: 2025, completed: 2024
2025-10-22 11:26:33,255 INFO: Filtering for season: 2024
2025-10-22 11:26:33,257 INFO: Processing data for prediction
Converting field PTS_home to int16 int64 79 155
Converting field AST_home to int16 int64 9 45
Converting field REB_home to int16 int64 23 73
Converting field PTS_away to int16 int64 67 162
Converting field AST_away to int16 int64 11 48
Converting field REB_away to int16 int64 24 66
2025-10-22 11:26:33,345 INFO: Making predictions

2025-10-22 11:26:33,542 INFO: Calculating daily running accuracy metrics
2025-10-22 11:26:33,551 INFO: Calculating team-specific daily running accuracy
2025-10-22 11:26:33,604 INFO: Calculating home/away daily running accuracy
2025-10-22 11:26:33,695 INFO: Calculating overall daily running accuracy
2025-10-22 11:26:33,880 INFO: Adding running accuracy metrics to games dataframe
2025-10-22 11:26:33,912 INFO: Calculating team-specific prediction accuracy (legacy method)
2025-10-22 11:26:33,945 INFO: Calculated team accuracy for 30 teams
2025-10-22 11:26:33,946 INFO: Processed 1302 completed games with 813 correct predictions
2025-10-22 11:26:33,950 INFO: Loading data from data\games_engineered.csv
2025-10-22 11:26:34,617 INFO: Successfully loaded data: 28323 rows, 245 columns
2025-10-22 11:26:34,620 INFO: Season selection -> upcoming: 2025, completed: 2024
2025-10-22 11:26:34,634 INFO: Filtering columns for dashboard
2025-10-22 11:26:34,638 INFO: Filtered dataframe from 257 to 22 columns

**Upload to Google Drive**

In [15]:

files_to_upload = [
    DATAPATH / 'games_dashboard.csv',
    DATAPATH / 'season_summary_stats.csv',
    DATAPATH / 'running_accuracy_metrics.csv'
]
upload_to_drive(files_to_upload, GOOGLE_FOLDER_ID)

2025-10-22 11:26:34,830 INFO: file_cache is only supported with oauth2client<4.0.0
Found 3 existing files in the folder
File 'games_dashboard.csv' updated successfully. File ID: 1g8Mc2SQafXApx8pwPBPhol2bDXiNOT46
File 'season_summary_stats.csv' updated successfully. File ID: 1zeqYZGhbfFj5kmj5ZRI1qXL2HKnU3B8S
File 'running_accuracy_metrics.csv' updated successfully. File ID: 18Hb24olk1Y4iroj55CEvDSxKXEGof4Ti


['1g8Mc2SQafXApx8pwPBPhol2bDXiNOT46',
 '1zeqYZGhbfFj5kmj5ZRI1qXL2HKnU3B8S',
 '18Hb24olk1Y4iroj55CEvDSxKXEGof4Ti']